# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Zadanie 1 - tokenizacja (12 pkt)

Jedną z nowoczesnych technik tokenizacji jest BPE - byte-pair encoding [1]. Technika ta polega na podzielenie słów na częste podsłowa (a'la morfemy?). W przeciwieństwie do podejść lingwistycznych, wymagających zwykle ręcznie napisanych reguł tworzenia morfemów czy nawet słowników lematów, BPE znajduje je heurystycznie poprzez wyznaczenie najczęstszych przylegających do siebie sekwencji znaków.

Algorytm przebiega w następujących krokach.
1. Podziel wszystkie słowa na symbole (początkowo pojedyncze znaki)
2. Wyznacz najczęściej występującą obok siebie parę symboli 
3. Stwórz nowy symbol będący konkatenacją dwóch najczęstszych symboli.

Uwaga 1: każde słowo zakończone jest specjalnym symbolem końca wyrazu.

Uwaga 2: tworzenie nowego symbolu nie powoduje usuniecie starego tj. zawsze jednym z możliwych symboli jest pojedynczy znak, ale jeśli można to stosujemy symbol dłuższy.

Przykład: korpus w którym występuje ,,ala'' 5 razy i ,,mama 10 razy''
1. Dzielimy słowa na symbole ,,a l a END'' ,,m a m a END''  gdzie END jest symbolem końca wyrazu.
2. Najczęstsza para obok siebie to ,,m a'' (20 razy)
3. Nowy symbol ,,ma''
4. Nowy podział ,,a l a END'' ,,ma ma END''
5. Najczęstsza para ,,ma ma'' (10 razy)
6. Nowy symbol ,,mama''
7. Nowy podział ,,a l a END'' ,,mama END''
8. itd.

W pliku ,,brown_clusters.tsv'' pierwsza kolumna to identyfikator skupienia (nie używamy w tym zadaniu), druga kolumna to wyrazy, a trzecia to ich liczności w pewnym korpusie tweetów. Zaimplementuj technikę BPE i przetesuj ją na tych słowach.

Parametrem algorytmu BPE jest `number_of_iterations` czyli liczba iteracji (łączeń symboli). Dodatkowo implementacja powinna mieć parametr `verbose`, którego wartość `True` powinna skutkować wypisywaniem na konsolę wykonywanych operacji (tj. łączeń).

[1] Sennrich, R., Haddow, B., and Birch, A. (2016). Neural machine translation of rare words with subword units. In ACL 2016.

In [ ]:
from nose.tools import assert_list_equal
from collections import defaultdict
import pandas as pd

def preform_bpe(brown_df: pd.DataFrame, number_of_iterations: int, verbose: bool = False):
  """
  Funckcja przyjmuje ramkę w formacie analogicznym do obiektu brown_df (wczytany wyżej)
   oraz liczbę iteracji.
  Wyjściem funkcji powinna być lista słów z poszczególnymi tokenami/symbolami oddzielonymi spacją.
  Za znak końca wyrazu przyjmij END.
  """
  brown_df.word = brown_df.word.astype(str)

  vocabulary = defaultdict(int)
  for _, row in brown_df.iterrows():
    # We add " END" to the end of each word to represent the end of the word
    word = ' '.join(row['word']) + " END"
    vocabulary[word] = row['count']

  for i in range(number_of_iterations):
    # A dictionary to hold pairs of characters and their frequency
    pairs = defaultdict(int)

    for (word, frequency) in vocabulary.items():
      symbols = word.split()
      for j in range(len(symbols) - 1):
        pairs[symbols[j], symbols[j + 1]] += frequency

    if not pairs: break

    most_frequent = max(pairs, key=pairs.get)
    new_vocabulary = defaultdict(int)
    merge_pattern = ' '.join(most_frequent)
    merged_value = ''.join(most_frequent)

    for word, frequency in vocabulary.items():
      new_word = word.replace(merge_pattern, merged_value)
      new_vocabulary[new_word] = frequency
    vocabulary = new_vocabulary

    if verbose: print(f"Iteration {i + 1}: Merging {most_frequent}")

  return list(vocabulary)

brown_df = pd.read_csv('./resources/brown_clusters.tsv', sep='\t', header=0, names=['cluster', 'word', 'count'])
number_of_iterations = 10
data = {'cluster': range(2), 'word': ['ala', 'mama'], 'count': [5, 10]}
df = pd.DataFrame(data, columns=['cluster', 'word', 'count'])
vocab = preform_bpe(df, 1)
assert_list_equal(vocab, ['a l a END', 'ma ma END'])
preform_bpe(brown_df, 50, verbose=True)

Test implementacji:

In [25]:
from nose.tools import assert_list_equal
data = {'cluster': range(2), 'word': ['ala', 'mama'], 'count': [5, 10]}
df = pd.DataFrame(data, columns=['cluster', 'word', 'count'])
vocab = preform_bpe(df, 1)
assert_list_equal(vocab, ['a l a END', 'ma ma END'])

Spraw aby Twoja implementacja wypisywała kolejne łączone ze sobą symbole (parametr `verbose`) i uruchom Twoją funkcję na np. 50 iteracji, obserwując jakie tokeny są tworzone.

In [26]:
preform_bpe(brown_df, 50, verbose=True)

TypeError: 'float' object is not iterable

- Jakie angielskie słowo jako pierwsze dostało swój własny token?

WPISZ TWOJĄ ODPOWIEDŹ TUTAJ

- Jakie są zalety korzystania z tokenizacji BPE w kontekście tworzenia reprezentacji do problemu klasyfikacji tekstu (problem OOV, odnieś się do k-gramów i n-gramów)? Jakie są zalety BPE w przypadku przetwarzania różny rodzajów języków (np. fleksyjne, aglutynacyjne)?

WPISZ TWOJĄ ODPOWIEDŹ TUTAJ

Wróć do implementacji i zakomentuj wypisywanie (funkcje print) informacji z funkcji `preform_bpe`.

# Zadanie 2 - klasyfikacja (15 pkt)

Poniższy kod powinien wczytać i ztokenizować zbiór danych dot. analizy wydźwięku. Jeśli nie masz biblioteki `nltk` musisz ją zainstalować.

In [ ]:
from helpers import DataSet
import nltk

training_set = DataSet(['./resources/tweets.txt'])

Poniżej znajdziesz przykład odczytu jednego tweeta z obiektu DataSet

In [ ]:
for i in training_set.tweets:
  print(i.text)
  print(i.tokens)
  print(i.clazz)
  break

Systemy IL często pracują z bardzo dużą liczbą cech, które są rzadkie np. cechy Bag-Of-Words, cechy n-gramowe itd. Powoduje to że klasyczna macierz zawierająca przykłady uczące ($n$) i cechy ($d$) rośnie do bardzo dużych rozmiarów ($nd$) nawet dla małych zbiorów uczących (w sensie liczby przykładów). Ponadto samo przechowywanie w pamięci słownika mapującego konkretne słowa/n-gramy na indeksy kolumn macierzy może być bardzo kosztowne pamięciowo przy dużych rozmiarach słownika.

Istnieje jednak technika, która pozwala nam na ominięcie tej przeszkody: haszowanie cech. Opis tej techniki znajdziesz na stronie:  https://en.wikipedia.org/wiki/Feature_hashing Jest ona też implementowana w obiekcie `sklearn.feature_extraction.FeatureHasher`. Zapoznaj się z opisem techniki i wykonaj poniższe polecenia.

- Wykorzystując haszowanie cech wytrenuj wybrany klasyfikator (najlepiej taki, który się szybko liczy) na zbiorze uczącym dla cech Bag-of-words (możesz też spróbować cechy n-gramowe). Możesz wykorzystać gotową tokenizację we właściwości `.tokens`.

In [ ]:
# WPISZ TWÓJ KOD TUTAJ
raise NotImplementedError()

- Stwórz wykres zależności wybranej miary klasyfikacji od wymiarów macierzy danych (chodzi o liczbę cech do których haszujemy cechy oryginalne). Wystarczy przetestować kilka (>=4) wybranych wartości na skali logarytmicznej.

In [ ]:
# WPISZ TWÓJ KOD TUTAJ
raise NotImplementedError()

 - Obserwując stworzony wykres - skomentuj. Jak dużo jakości klasyfikacji się traci (albo zyskuje?) korzystając z mniejszej liczby haszowanych cech? Często klasyfikatory bardzo dobrze działają nawet przy liczbie haszowanych cech dla których na pewno istnieją konflikty cech oryginalnych - jak myślisz dlaczego? (Pomyśl o interpretacji takich skonfliktowanych cech).

WPISZ TWOJĄ ODPOWIEDŹ TUTAJ

 - W poprzednim zadaniu wczytałeś wynik grupowania Browna do pamięci. Wytrenuj klasyfikator na reprezentacji ,,Bag-of-clusters'' tj. w kolumnach zamiast słów/n-gramów będziesz miał grupy.

In [ ]:
# WPISZ TWÓJ KOD TUTAJ
raise NotImplementedError()

- Podsumuj eksperymenty: poznałeś dwie możliwości ograniczenia liczby cech - zastąpienie słów ich grupami i haszowanie cech. Jakie są wady i zalety obydwu podejść?

WPISZ TWOJĄ ODPOWIEDŹ TUTAJ